# Domain Randomizationの理論

参考
* [Understanding Domain Randomization for Sim-to-real Transfer](https://arxiv.org/abs/2110.03239)

Domain RandomizationはRLで頻繁に使われる技術ですが，なぜうまくいくのか？についての理論はそんなに多くありません．
つまり，「シミュレータでDomain randomizationによって学習させた方策は，デプロイした際，そのSub-optimality gapのバウンドは良くなるのか？」について見ていきます．

表記
* MDP：$\mathcal{M}=\left(\mathcal{S}, \mathcal{A}, P, R, H, s_1\right)$
* $\operatorname{traj}_h=\left\{\left(s_1, a_1, s_2, a_2, \cdots, s_h\right) \mid s_i \in \mathcal{S}, a_i \in \mathcal{A}, i \in[h]\right\}$
    * $h$ステップ目までにありえる全ての軌跡の集合
* history-dependentな方策の集合：$\Pi$
    * 軌跡から行動の分布へのマップ：$\left\{\pi_h: \operatorname{traj}_h \rightarrow \Delta(\mathcal{A})\right\}_{h \in[H]}$   
* 状態価値関数：$V_{\mathcal{M}, h}^\pi(s)=\mathbb{E}_{\mathcal{M}, \pi}\left[\sum_{t=h}^H R\left(s_t, a_t\right) \mid s_h=s\right]$
* MDPの集合：$\mathcal{U}$
    * それぞれのMDPを$\mathcal{M}=\left(\mathcal{S}, \mathcal{A}, P_{\mathcal{M}}, R, H, s_1\right)$として，独自のTransition $P_\mathcal{M}$ を持つとします．
    * $\mathcal{U}$の中に真のMDP $\mathcal{M}^*$（実世界のダイナミクス）があるとします．
    * $\nu$を$\mathcal{U}$上の分布とします．
* $(\mathcal{U}, \nu)$のことをLatent Markov decision processと呼びます．

$\pi$をシミュレーションで学習させた方策とします．Sim-to-realでは，次のギャップが小さいことが望ましいです．

$$
\operatorname{Gap}(\pi)=V_{\mathcal{M}^*, 1}^*\left(s_1\right)-V_{\mathcal{M}^*, 1}^\pi\left(s_1\right)
$$

Sim-to-realでは，$\mathcal{U}$を何かしらの方法でパラメータ化し，そのパラメータをランダマイズすることで$\mathcal{U}$からMDPをサンプルします（つまり，$\nu$からサンプルします）．
話を簡単にするために，次の定義を導入します．

---

**Domain Randomization Oracle**

$\mathcal{U}$と$\mathcal{U}$上の一様分布$\nu$に対して，domain randomization oracleは次の$\pi_{D R}^*$を返します．
$$
\pi_{D R}^*=\underset{\pi \in \Pi}{\arg \max } \mathbb{E}_{\mathcal{M} \sim \nu} V_{\mathcal{M}, 1}^\pi\left(s_1\right)
$$

Sim-to-real gapは$\operatorname{Gap}\left(\pi_{\mathrm{DR}}^*, \mathcal{U}\right)$に相当します．

---


解析のために，次の仮定を導入します．

---

**Communicating MDPs**

仮定：全ての$\mathcal{M}\in \mathcal{U}$の半径は$D$でバウンドされている．

つまり，stationaryな方策$\pi: \mathcal{S} \rightarrow \mathcal{A}$として，$T\left(s^{\prime} \mid \mathcal{M}, \pi, s\right)$を最初に$s'$がふまれるタイムステップの確率変数だとすると，
$$
\max _{s \neq s^{\prime} \in \mathcal{S}} \min _{\pi: \mathcal{S} \rightarrow \mathcal{A}} \mathbb{E}\left[T\left(s^{\prime} \mid \mathcal{M}, \pi, s\right)\right] \leq D
$$
が成立している状況です．

---

ちなみにこの仮定がなければ，$\operatorname{Gap}(\pi^*_{D R})=\Omega(H)$になる$\mathcal{U}$が存在します．（論文参照）

## $\mathcal{U}$が有限のとき

$\mathcal{U}$が有限であり，また，その半径が$M$である場合を考えましょう．また，$\mathcal{U}$の中のMDPは次のように判別可能であるとします．

---

**$\delta$-separated MDP**

全ての$\mathcal{M}_1, \mathcal{M}_2 \in \mathcal{U}$について，
$$
\left\|\left(P_{\mathcal{M}_1}-P_{\mathcal{M}_2}\right)(\cdot \mid s, a)\right\|_1 \geq \delta
$$
を満たすような$(s, a)$が存在する．

---

このとき，次のSim-to-real gapが任意の$\mathcal{M}$について成立します．

$$
\operatorname{Gap}\left(\pi_{D R}^*\right)=O\left(\frac{D M^3 \log (M H) \log ^2(S M H / \delta)}{\delta^4}\right)
$$

また，$\delta$-separated MDPの仮定がなくても，次のGapが保証されます．

$$
\operatorname{Gap}\left(\pi_{D R}^*\right)=O\left(D \sqrt{M^3 H \log (M H)}\right)
$$

ちなみに$\sqrt{H}$のGapは不可避です．

**証明**

証明は基本的に次の戦略で行います．
何らかのsim-to-real gapが小さい基本の方策$\hat{\pi} \in \Pi$が存在する場合に，$\pi_{\mathrm{DR}}^*$のgapがそんなに大きくならないことを示します．
そこで，次の定理が便利です．

---

任意の$\mathcal{M}$について，
$$
V_{\mathcal{M}, 1}^*\left(s_1\right)-V_{\mathcal{M}, 1}^{\hat{\pi}}\left(s_1\right) \leq C
$$
が成立するような方策$\hat{\pi}$が存在するとします．このとき，$\mathcal{U}$のサイズが$M$なら，
$\operatorname{Gap}\left(\pi_{D R}^*\right) \leq M C$が成立します．

証明は省略します．（論文参照）

---

まずSeparation conditionが成立する場合について証明します．

ここで，history dependentな方策は何らかのアルゴリズムであるとみなすことができることに注意しましょう．
つまり，軌跡を入力されて行動の分布を返すアルゴリズムです．そのようなアルゴリズムとして，次を考えます．

![DM-algo](figs/domain-random.png)
![DM-algo2](figs/domain-random2.png)

Algorithm１は環境を探索して真の$\mathcal{M}^*$を探し当てることを目標にします．$\mathcal{D}=\mathcal{U}$として初期化します．

1. まず，$\mathcal{D}$の中からランダムに$\mathcal{M}_1$と$\mathcal{M}_2$をサンプルします．
2. $\mathcal{M}_1$と$\mathcal{M}_2$は既知なので，$\left(s_0, a_0\right)=\arg \max _{(s, a) \in \mathcal{S} \times \mathcal{A}}\left\|\left(P_{\mathcal{M}_1}-P_{\mathcal{M}_2}\right)(\cdot \mid s, a)\right\|_1$を見つけます．
3. 続いて，Algorithm 2を$(s_0, a_0)$に対して走らせます．
    * Algorithm 2は$(s_0, a_0)$を訪れる回数が$n_0$に達するまで，全てのMDPを走査します．最終的に$(s_0, a_0)$の次の状態$s'$のサンプルを$n_0$個集め，それを返します．返ってきたものを$\mathcal{H}_{\mathcal{M}_1, \mathcal{M}_2}$とします．
4. もし次のどちらかが成立する場合は$\mathcal{D}$から$\mathcal{M}_2$を排除します．そうでない場合は$\mathcal{M}_1$を排除します．
   1. $\exists s^{\prime} \in \mathcal{H}_{\mathcal{M}_1, \mathcal{M}_2}, P_{\mathcal{M}_2}\left(s^{\prime} \mid s_0, a_0\right)=0$，つまり，$\mathcal{H}_{\mathcal{M}_1, \mathcal{M}_2}$に入っている状態のうち，$\mathcal{M}_2$ではそこに遷移しないような状態が存在する場合です．
   2. $\prod_{s^{\prime} \in \mathcal{H}_{\mathcal{M}_1, \mathcal{M}_2}} \frac{P_{\mathcal{M}_1}\left(s^{\prime} \mid s_0, a_0\right)}{P_{\mathcal{M}_2}\left(s^{\prime} \mid s_0, a_0\right)} \geq 1$，つまり，$s'$に遷移する確率が$\mathcal{M}_1$のほうが高い場合です．
5. 以上を繰り返し，最後に残ったMDP $\hat{M}$の最適方策を返します．

このようなAlgorithm 1によって表現される方策 $\hat{\pi}$は，次のsim-to-real gapを保証します．

$$
V_{\mathcal{M}, 1}^*\left(s_1\right)-V_{\mathcal{M}, 1}^{\hat{\pi}}\left(s_1\right) \leq O\left(\frac{D M^2 \log (M H) \log ^2(S M H / \delta)}{\delta^4}\right)
$$

TODO: 残りの証明